In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [2]:
# Load and preprocess data 
# Load the data
train_df = pd.read_csv('csv/pokemon_train.csv')
test_df = pd.read_csv('csv/pokemon_test.csv')

# Encode categorical variables
le_gen = LabelEncoder()
le_primary = LabelEncoder()
le_secondary = LabelEncoder()

train_df['Generation'] = le_gen.fit_transform(train_df['Generation'])
train_df['Primary_Type'] = le_primary.fit_transform(train_df['Primary_Type'])
train_df['Secondary_Type'] = le_secondary.fit_transform(train_df['Secondary_Type'].fillna('None'))

test_df['Generation'] = le_gen.transform(test_df['Generation'])
test_df['Primary_Type'] = le_primary.transform(test_df['Primary_Type'])
test_df['Secondary_Type'] = le_secondary.transform(test_df['Secondary_Type'].fillna('None'))

# Convert to one-hot encoding
gen_classes = len(le_gen.classes_)
primary_classes = len(le_primary.classes_)
secondary_classes = len(le_secondary.classes_)

y_train_gen = to_categorical(train_df['Generation'], num_classes=gen_classes)
y_train_primary = to_categorical(train_df['Primary_Type'], num_classes=primary_classes)
y_train_secondary = to_categorical(train_df['Secondary_Type'], num_classes=secondary_classes)

y_test_gen = to_categorical(test_df['Generation'], num_classes=gen_classes)
y_test_primary = to_categorical(test_df['Primary_Type'], num_classes=primary_classes)
y_test_secondary = to_categorical(test_df['Secondary_Type'], num_classes=secondary_classes)

In [3]:
import os
from PIL import Image
import numpy as np

def load_and_preprocess_image(row, base_dir='pokemon_train_images', target_size=(224, 224)):
    name = row['Name']
    gen = int(row['Generation'])
    
    # Construct the path to the image
    # note the image paths
    img_path = os.path.join(base_dir, f'gen_{gen+1}.0', f'{name}.png')
    
    try:
        # Open the image
        img = Image.open(img_path).convert('RGBA')
        
        # Create a white background
        background = Image.new('RGBA', img.size, (255, 255, 255, 255))
        
        # Alpha composite the image with the background
        img = Image.alpha_composite(background, img)
        
        # Make white background transparent
        data = np.array(img)
        r, g, b, a = data.T
        white_areas = (r == 255) & (g == 255) & (b == 255)
        data[..., :][white_areas.T] = (255, 255, 255, 0)
        
        # Convert back to PIL Image and resize
        img = Image.fromarray(data)
        img = img.resize(target_size)
        
        # Convert to RGB (removing alpha channel) and normalize
        img_array = np.array(img.convert('RGB')) / 255.0
        
        return img_array
    except Exception as e:
        print(f"Error processing {name}: {str(e)}")
        return None

# Apply the function to your DataFrame and handle None values
X_train = []
for _, row in train_df.iterrows():
    img = load_and_preprocess_image(row)
    if img is not None:
        X_train.append(img)

X_test = []
for _, row in test_df.iterrows():
    img = load_and_preprocess_image(row, base_dir='pokemon_test_images')
    if img is not None:
        X_test.append(img)

# Convert to numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

X_train shape: (820, 224, 224, 3)
X_test shape: (205, 224, 224, 3)


In [4]:
# Model architecture

# Get paths from manually downlaoded weights
weights_path = '.keras/models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model = ResNet50(weights=weights_path, include_top=False, input_shape=(224, 224, 3))

x = GlobalAveragePooling2D()(base_model.output)

# Generation branch
gen_output = Dense(gen_classes, activation='softmax', name='generation')(x)

# Primary Type branch
primary_output = Dense(primary_classes, activation='softmax', name='primary_type')(x)

# Secondary Type branch
secondary_output = Dense(secondary_classes, activation='softmax', name='secondary_type')(x)

model = Model(inputs=base_model.input, outputs=[gen_output, primary_output, secondary_output])

In [9]:
# Model compilation
model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'generation': 'categorical_crossentropy',
                    'primary_type': 'categorical_crossentropy',
                    'secondary_type': 'categorical_crossentropy'},
              metrics={'generation': 'accuracy',
                       'primary_type': 'accuracy',
                       'secondary_type': 'accuracy'})

history = model.fit(X_train, 
                    {'generation': y_train_gen, 
                     'primary_type': y_train_primary, 
                     'secondary_type': y_train_secondary},
                    validation_split=0.2,
                    epochs=50,
                    batch_size=32)

Epoch 1/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 95s 4s/step - generation_accuracy: 0.2368 - generation_loss: 2.2180 - loss: 7.7234 - primary_type_accuracy: 0.1669 - primary_type_loss: 2.9904 - secondary_type_accuracy: 0.3724 - secondary_type_loss: 2.5145 - val_generation_accuracy: 0.1585 - val_generation_loss: 379.6910 - val_loss: 1219.6740 - val_primary_type_accuracy: 0.0732 - val_primary_type_loss: 596.7734 - val_secondary_type_accuracy: 0.0183 - val_secondary_type_loss: 175.9896
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 83s 4s/step - generation_accuracy: 0.3579 - generation_loss: 1.7758 - loss: 6.0218 - primary_type_accuracy: 0.2830 - primary_type_loss: 2.2902 - secondary_type_accuracy: 0.4772 - secondary_type_loss: 1.9577 - val_generation_accuracy: 0.0976 - val_generation_loss: 244.1945 - val_loss: 572.8776 - val_primary_type_accuracy: 0.0793 - val_primary_type_loss: 180.0179 - val_secondary_type_accuracy: 0.0427 - val_secondary_type_loss: 139.7545
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 78s 4s/st

KeyboardInterrupt: 

In [8]:
# Data augmentation and model training

# print("X_train shape:", X_train.shape)
# print("y_train_gen shape:", y_train_gen.shape)
# print("y_train_primary shape:", y_train_primary.shape)
# print("y_train_secondary shape:", y_train_secondary.shape)

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Combine target variables
y_train = [y_train_gen, y_train_primary, y_train_secondary]
y_test = [y_test_gen, y_test_primary, y_test_secondary]

# Further training with data augmentation
augmented_history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    steps_per_epoch=len(X_train) // 32,
    epochs=50,
    validation_data=(X_test, y_test)
)

X_train shape: (820, 224, 224, 3)
y_train_gen shape: (820, 9)
y_train_primary shape: (820, 18)
y_train_secondary shape: (820, 19)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (3, 820) + inhomogeneous part.

In [ ]:
# Model evaluation
test_loss, test_gen_loss, test_primary_loss, test_secondary_loss, test_gen_acc, test_primary_acc, test_secondary_acc = model.evaluate(X_test, [y_test_gen, y_test_primary, y_test_secondary])

print(f"Test Generation Accuracy: {test_gen_acc:.4f}")
print(f"Test Primary Type Accuracy: {test_primary_acc:.4f}")
print(f"Test Secondary Type Accuracy: {test_secondary_acc:.4f}")

In [ ]:
# Visualize training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(history.history['generation_accuracy'], label='Train')
plt.plot(history.history['val_generation_accuracy'], label='Validation')
plt.title('Generation Accuracy')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(history.history['primary_type_accuracy'], label='Train')
plt.plot(history.history['val_primary_type_accuracy'], label='Validation')
plt.title('Primary Type Accuracy')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(history.history['secondary_type_accuracy'], label='Train')
plt.plot(history.history['val_secondary_type_accuracy'], label='Validation')
plt.title('Secondary Type Accuracy')
plt.legend()

plt.tight_layout()
plt.show()